In [43]:
using CSV, DataFrames
# D = demand matrix (day of the week)
# n locations
# W = foot-traffic matrix (for each location, foot traffic for day of the week)

# D = CSV.read("electricity/demand.csv", DataFrame; header=true)[:, 2]#7x1
D = vec([602.4285714*1000,602.4285714*1000,602.4285714*1000,602.4285714*1000,602.4285714*1000,602.4285714*1000,602.4285714*1000]) #7x1

mean_counts_df = CSV.read("ped_traffic/mean_counts.csv", DataFrame; header=true)
W = 100 .* mean_counts_df[:, 3:9]|> Matrix #110x7 (200 steps per pedestrain)
location_ids = mean_counts_df[:, 1] # location IDs

# mean_counts_df2 = CSV.read("ped_traffic/mean_counts_dist.csv", DataFrame; header=true)
mean_counts_df2 = CSV.read("ped_traffic/mean_counts_dist_ss.csv", DataFrame; header=true)
W2 = 100 .* mean_counts_df2[:, 3:9]|> Matrix #44x7
d = mean_counts_df2[:, 10] #44x1
location_ids2 = mean_counts_df2[:, 1] # location IDs

println(size(D))
println(size(W))
println(size(W2))
println(size(d))

(7,)
(110, 7)
(44, 7)
(44,)


# Number of Locations Minimization Optimization

In [ ]:
using JuMP
using HiGHS
using LinearAlgebra

percent_demand = 0.00001
watts_converted = 0.1

function optimize_installations(D::Vector{Float64}, W::Matrix{Float64})

    n, T = size(W)  # n locations, T = 7 days
    @assert length(D) == T "Demand vector D must have length 7"

    model = Model(HiGHS.Optimizer)
    
    set_silent(model)

    @variable(model, x[1:n], Bin) # int decision vars

    @objective(model, Min, sum(x[i] for i in 1:n)) # objective func

    for t in 1:T # constraint each day
        @constraint(model, sum(W[i, t] * x[i] * watts_converted for i in 1:n) ≥ percent_demand * D[t])
    end

    optimize!(model) # solve model

    status = termination_status(model)
    if status != MOI.OPTIMAL
        println("Warning: Solver returned status $status")
    end

    chosen_locations = findall(i -> value(x[i]) > 0.5, 1:n)
    min_num_locations = length(chosen_locations)

    return min_num_locations, chosen_locations, model
end

In [ ]:
min_locs, chosen, model = optimize_installations(D, W)
println("Minimum number of locations needed: ", min_locs)
println("Chosen location indices: ", chosen)

In [ ]:
min_locs, chosen, model = optimize_installations(D, W2)
println("Minimum number of locations needed: ", min_locs)
println("Chosen location indices: ", chosen)

In [ ]:
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
energy_produced = zeros(7)
energy_needed = percent_demand .* D

println(chosen)

for t in 1:7
    for i in chosen
        energy_produced[t] += W[i, t] * 0.1
        # energy_produced[t] += W2[i, t] * 0.1
    end
end

energy_comparison = DataFrame(
    Day = days,
    Energy_Produced = energy_produced,
    Energy_Needed = energy_needed,
    Surplus = energy_produced .- energy_needed,
)

println("=" ^ 70)
println("Energy Production vs Demand Analysis")
println("=" ^ 70)
println(energy_comparison)
println()


In [ ]:
total_foot_traffic = [sum(W[i, :]) for i in 1:size(W, 1)]

println(chosen)

location_traffic = DataFrame(
    Index = 1:size(W, 1),
    Total_Foot_Traffic = total_foot_traffic
)
sort!(location_traffic, :Total_Foot_Traffic, rev=true)

println("=" ^ 70)
println("Top 10 Locations by Total Foot Traffic")
println("=" ^ 70)
println(first(location_traffic, 10))

# Distance Minimization Optimization

In [28]:
print(D)

[602428.5714, 602428.5714, 602428.5714, 602428.5714, 602428.5714, 602428.5714, 602428.5714]

In [46]:
using JuMP
using HiGHS
using LinearAlgebra

percent_demand = 0.25
watts_converted = 0.1

function optimize_installations_dist(D::Vector{Float64}, W::Matrix{Float64}, d::Vector{Float64}, max_loc)

    n, T = size(W)  # n locations, T = 7 days
    @assert length(D) == T "Demand vector D must have length 7"
    @assert length(d) == n "Distance vector d must have length n"

    model = Model(HiGHS.Optimizer)
    
    set_silent(model)

    @variable(model, x[1:n], Bin) # int decision vars

    @objective(model, Min, sum((d[i]*d[i] + d[i]) * x[i] for i in 1:n))

    @constraint(model, sum(x[i] for i in 1:n) ≤ max_loc)
    for t in 1:T
        @constraint(model, sum(W[i, t] * x[i] * watts_converted for i in 1:n) ≥ percent_demand * D[t])
    end

    optimize!(model)

    status = termination_status(model)
    if status != MOI.OPTIMAL
        println("Warning: Solver returned status $status")
    end

    chosen_locations = findall(i -> value(x[i]) > 0.5, 1:n)
    total_distance = sum(d[i] for i in chosen_locations)
    num_locations = length(chosen_locations)

    return num_locations, chosen_locations, total_distance, model
end

optimize_installations_dist (generic function with 1 method)

In [47]:
num_locs, chosen, total_dist, model = optimize_installations_dist(D, W2, d, 110)
total_objective = sum(d[i]*d[i]+d[i] for i in chosen)

println("Number of locations selected: ", num_locs)
println("Chosen location indices: ", chosen)
println("Total distance to South Station: ", total_dist, " miles")
println("Total objective value: ", total_objective)

Number of locations selected: 7
Chosen location indices: [5, 6, 9, 10, 20, 33, 34]
Total distance to South Station: 10.729415370677605 miles
Total objective value: 31.803697092999386


In [46]:
num_locs, chosen, total_dist, model = optimize_installations_dist(D, W2, d, 4)
total_objective = sum(d[i]*d[i]+d[i] for i in chosen)

println("Number of locations selected: ", num_locs)
println("Chosen location indices: ", chosen)
println("Total distance to City Hall: ", total_dist, " miles")
println("Total objective value: ", total_objective)

Number of locations selected: 4
Chosen location indices: [6, 9, 10, 34]
Total distance to City Hall: 5.368448838796317 miles
Total objective value: 14.91512325402067


In [42]:
num_locs, chosen, total_dist, model = optimize_installations_dist(D, W2, d, 3)
total_objective = sum(d[i]*d[i]+d[i] for i in chosen)

println("Number of locations selected: ", num_locs)
println("Chosen location indices: ", chosen)
println("Total distance to City Hall: ", total_dist, " miles")
println("Total objective value: ", total_objective)

Number of locations selected: 32
Chosen location indices: [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44]
Total distance to City Hall: 3931.8413985485654 miles
Total objective value: 1.0948946488273254e7


In [47]:
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
energy_produced = zeros(7)
energy_needed = percent_demand .* D

println(chosen)

for t in 1:7
    for i in chosen
        # energy_produced[t] += W[i, t] * 0.1
        energy_produced[t] += W2[i, t] * 0.1
    end
end

energy_comparison = DataFrame(
    Day = days,
    Energy_Produced = energy_produced,
    Energy_Needed = energy_needed,
    Surplus = energy_produced .- energy_needed,
)

println("=" ^ 70)
println("Energy Production vs Demand Analysis")
println("=" ^ 70)
println(energy_comparison)
println()


[6, 9, 10, 34]
Energy Production vs Demand Analysis
7×4 DataFrame
 Row │ Day        Energy_Produced  Energy_Needed   Surplus  
     │ String     Float64          Float64         Float64  
─────┼──────────────────────────────────────────────────────
   1 │ Monday            239470.0       2.28841e5  10629.1
   2 │ Tuesday           239470.0  236349.0         3121.02
   3 │ Wednesday         239470.0       2.38229e5   1241.37
   4 │ Thursday          239470.0       2.38003e5   1467.19
   5 │ Friday            239470.0       2.32287e5   7183.19
   6 │ Saturday          239470.0       2.10892e5  28578.2
   7 │ Sunday            239470.0       2.1237e5   27100.3



In [8]:
total_foot_traffic = [sum(W2[i, :]) for i in 1:size(W2, 1)]

println(chosen)
location_traffic = DataFrame(
    Index = 1:size(W2, 1),
    Total_Foot_Traffic = total_foot_traffic
)
sort!(location_traffic, :Total_Foot_Traffic, rev=true)

println("=" ^ 70)
println("Top 10 Locations by Total Foot Traffic")
println("=" ^ 70)
println(first(location_traffic, 10))

[10]
Top 10 Locations by Total Foot Traffic
10×2 DataFrame
 Row │ Index  Total_Foot_Traffic 
     │ Int64  Float64            
─────┼───────────────────────────
   1 │     6           6.9216e6
   2 │    10           4.844e6
   3 │     9           4.0691e6
   4 │    24           2.91375e6
   5 │     8           2.8399e6
   6 │    20           2.2155e6
   7 │    33           1.84485e6
   8 │     7           1.001e6
   9 │    36      947800.0
  10 │    34      928200.0


In [9]:
selected_locations = DataFrame(
    Index = 1:size(W2, 1),
    Distance_miles = d
)

println(chosen)
sort!(selected_locations, :Distance_miles)

println("=" ^ 70)
println("Selected Locations Sorted by Distance to City Hall")
println("=" ^ 70)
println(first(selected_locations, 10))


[10]
Selected Locations Sorted by Distance to City Hall
10×2 DataFrame
 Row │ Index  Distance_miles 
     │ Int64  Float64        
─────┼───────────────────────
   1 │    10        0.357348
   2 │     2        0.380489
   3 │    15        0.662542
   4 │    16        0.662542
   5 │    14        0.693014
   6 │     1        0.854314
   7 │     5        0.854314
   8 │    34        1.252
   9 │     6        1.25263
  10 │    39        1.42072


# Robust Opt

### Element-Wise Robustness

In [10]:
using JuMP
using HiGHS
using LinearAlgebra
using Statistics

percent_demand = 0.00001
watts_converted = 0.1

function optimize_installations_dist_robust(
    D_mean::Vector{Float64},      # Mean demand vector
    D_std::Vector{Float64},        # Standard deviation of demand
    W_mean::Matrix{Float64},       # Mean traffic matrix
    W_std::Matrix{Float64},        # Standard deviation of traffic matrix
    d::Vector{Float64},            # Distance vector
    k = 2.0,              # Uncertainty parameter (k standard deviations)
    max_loc = 1000               # Maximum number of locations
)

    n, T = size(W_mean)  # n locations, T = 7 days
    @assert length(D_mean) == T "Demand vector must have length 7"
    @assert length(D_std) == T "Demand std vector must have length 7"
    @assert size(W_std) == size(W_mean) "W_std must have same size as W_mean"
    @assert length(d) == n "Distance vector d must have length n"

    model = Model(HiGHS.Optimizer)
    set_silent(model)

    @variable(model, x[1:n], Bin) # Binary decision variables

    # Objective: minimize e^distance - 1 for selected locations
    @objective(model, Min, sum((d[i]*d[i] + d[i]) * x[i] for i in 1:n))

    # Constraint: maximum locations can be selected
    @constraint(model, sum(x[i] for i in 1:n) ≤ max_loc)

    # Robust constraint: energy production must meet demand each day
    # For worst-case scenario: minimum energy production ≥ maximum demand
    # Worst-case energy production: W_mean - k*W_std (conservative)
    # Worst-case demand: D_mean + k*D_std (conservative)
    for t in 1:T
        # Robust constraint: must satisfy for worst-case (minimum traffic, maximum demand)
        @constraint(model, 
            sum((W_mean[i, t] - k * W_std[i, t]) * x[i] * watts_converted for i in 1:n) 
            ≥ percent_demand * (D_mean[t] + k * D_std[t])
        )
    end

    optimize!(model) # solve model

    status = termination_status(model)
    if status != MOI.OPTIMAL
        println("Warning: Solver returned status $status")
    end

    chosen_locations = findall(i -> value(x[i]) > 0.5, 1:n)
    total_distance = sum(d[i] for i in chosen_locations)
    num_locations = length(chosen_locations)

    return num_locations, chosen_locations, total_distance, model
end


optimize_installations_dist_robust (generic function with 3 methods)

In [11]:
# future work: compute D_mean, D_std, W_mean, W_std from historical data
D_mean_robust = D
D_std_robust = 0.1 .* D_mean_robust # Assume 10% coefficient of variation for demand (std = 0.1 * mean)

W_mean_robust = W2
W_std_robust = max.(0.15 .* W_mean_robust, 0.01) # Assume 15% coefficient of variation for traffic (std = 0.15 * mean); ensure non-negative standard deviations

44×7 Matrix{Float64}:
   9043.75     9467.5     6997.5   …    3877.5     9043.75     9043.75
   2445.0      2445.0     2445.0        2445.0     2445.0      2445.0
  14430.0     23790.0    14235.0       14430.0    10485.0      9210.0
  16170.0     16170.0    16170.0       16170.0    11580.0     16170.0
  15715.0     15532.5    10985.0       15715.0    15715.0     15715.0
 148320.0    148320.0   148320.0   …  148320.0   148320.0    148320.0
  21450.0     24720.0    21450.0       21450.0    21450.0     21450.0
  60855.0     60855.0    60855.0       60855.0    60855.0     60855.0
  87195.0     87195.0    87195.0       87195.0    87195.0     87195.0
 103800.0    103800.0   103800.0      103800.0   103800.0    103800.0
  18615.0     18615.0    18615.0   …   18615.0    18615.0     18615.0
   4800.0      4800.0     4800.0        4800.0     4800.0      4800.0
   3300.0      3300.0     3300.0        3300.0     3300.0      3300.0
      ⋮                            ⋱                  ⋮      
  395

In [12]:
num_locs_robust, chosen_robust, total_dist_robust, model_robust = 
    optimize_installations_dist_robust(
        D_mean_robust, D_std_robust, 
        W_mean_robust, W_std_robust, 
        d, 
        1 # set uncertainty parameter k (e.g., k=2.0 means protect against 2 standard deviations)
    )
total_objective_robust = sum(d[i]*d[i] + d[i] for i in chosen_robust)

println("Robust Optimization Results:")
println("Number of locations selected: ", num_locs_robust)
println("Chosen location indices: ", chosen_robust)
println("Total distance to City Hall: ", total_dist_robust, " miles")
println("Total objective value: ", total_objective_robust)
println()

Robust Optimization Results:
Number of locations selected: 1
Chosen location indices: [10]
Total distance to City Hall: 0.3573483661354215 miles
Total objective value: 0.48504622091507676



In [13]:
num_locs_robust, chosen_robust, total_dist_robust, model_robust = 
    optimize_installations_dist_robust(
        D_mean_robust, D_std_robust, 
        W_mean_robust, W_std_robust, 
        d, 
        2 # set uncertainty parameter k (e.g., k=2.0 means protect against 2 standard deviations)
    )
total_objective_robust = sum(d[i]*d[i] + d[i] for i in chosen_robust)

println("Robust Optimization Results:")
println("Number of locations selected: ", num_locs_robust)
println("Chosen location indices: ", chosen_robust)
println("Total distance to City Hall: ", total_dist_robust, " miles")
println("Total objective value: ", total_objective_robust)
println()

Robust Optimization Results:
Number of locations selected: 1
Chosen location indices: [10]
Total distance to City Hall: 0.3573483661354215 miles
Total objective value: 0.48504622091507676



In [14]:
num_locs_robust, chosen_robust, total_dist_robust, model_robust = 
    optimize_installations_dist_robust(
        D_mean_robust, D_std_robust, 
        W_mean_robust, W_std_robust, 
        d, 
        1, # set uncertainty parameter k (e.g., k=2.0 means protect against 2 standard deviations)
        7 # set max_loc param
    )
total_objective_robust = sum(d[i]*d[i] + d[i] for i in chosen_robust)

println("Robust Optimization Results:")
println("Number of locations selected: ", num_locs_robust)
println("Chosen location indices: ", chosen_robust)
println("Total distance to City Hall: ", total_dist_robust, " miles")
println("Total objective value: ", total_objective_robust)
println()

Robust Optimization Results:
Number of locations selected: 1
Chosen location indices: [10]
Total distance to City Hall: 0.3573483661354215 miles
Total objective value: 0.48504622091507676



In [15]:
num_locs_robust, chosen_robust, total_dist_robust, model_robust = 
    optimize_installations_dist_robust(
        D_mean_robust, D_std_robust, 
        W_mean_robust, W_std_robust, 
        d, 
        1, # set uncertainty parameter k (e.g., k=2.0 means protect against 2 standard deviations)
        6 # set max_loc param
    )
total_objective_robust = sum(d[i]*d[i] + d[i] for i in chosen_robust)

println("Robust Optimization Results:")
println("Number of locations selected: ", num_locs_robust)
println("Chosen location indices: ", chosen_robust)
println("Total distance to City Hall: ", total_dist_robust, " miles")
println("Total objective value: ", total_objective_robust)
println()

Robust Optimization Results:
Number of locations selected: 1
Chosen location indices: [10]
Total distance to City Hall: 0.3573483661354215 miles
Total objective value: 0.48504622091507676



### Global Robustness

In [16]:
using JuMP
using Gurobi
using LinearAlgebra

function optimize_installations_dist_robust_global(
    D_mean::Vector{Float64},      
    D_std::Vector{Float64},       
    W_mean::Matrix{Float64},      
    W_std::Matrix{Float64},       
    d::Vector{Float64},           
    k = 2.0,                       
    max_loc = 1000
)

    n, T = size(W_mean)

    model = Model(Gurobi.Optimizer)
    set_silent(model)
    set_optimizer_attribute(model, "OutputFlag", 0)
    set_optimizer_attribute(model, "LogToConsole", 0)

    @variable(model, x[1:n], Bin)
    @variable(model, aux[1:T] ≥ 0)

    @objective(model, Min, sum((d[i] * d[i] + d[i]) * x[i] for i in 1:n))
    @constraint(model, sum(x) ≤ max_loc)

    for t in 1:T
        # SOCP constraint for uncertainty budget
        @constraint(model,
            [aux[t];  [W_std[i,t] * x[i] for i in 1:n]...] in SecondOrderCone()
        )

        @constraint(model,
            sum(W_mean[i,t] * x[i] for i in 1:n)
            - k * sqrt(n) * aux[t]
            ≥ percent_demand * (D_mean[t] + k * D_std[t])
        )
    end

    optimize!(model)

    chosen = findall(i -> value(x[i]) > 0.5, 1:n)
    total_dist = sum(d[i] for i in chosen)

    return length(chosen), chosen, total_dist, model
end


optimize_installations_dist_robust_global (generic function with 3 methods)

In [17]:
num_locs_g, chosen_g, total_dist_g, model_g =
    optimize_installations_dist_robust_global(
        D_mean_robust, D_std_robust,
        W_mean_robust, W_std_robust,
        d, 
        1,  # k
        6   # max_loc
    )

println("Robust 2 Global Results:")
println("Locations selected: ", num_locs_g)
println("Chosen: ", chosen_g)
println("Total distance: ", total_dist_g)


Set parameter Username
Set parameter LicenseID to value 2703538


LoadError: Gurobi Error 10009: Version number is 13.0, license is for version 12.0

### Plot Results

In [ ]:
using Plots
using Statistics

ENV["HIGHS_OPTIONS"] = "store_basis=off"

# Range of k to test
k_values = 0:0.2:2.4

# Storage vectors
objectives = Float64[]
prob_feasible = Float64[]

# Number of Monte Carlo samples for feasibility estimation
N_MC = 2000

println("Running robust optimization sweep...\n")

for k in k_values
    println("Testing k = ", k)

    # Solve optimization for this k
    num_locs, chosen, total_dist, model =
        optimize_installations_dist_robust(
            D_mean_robust, D_std_robust,
            W_mean_robust, W_std_robust,
            d, k
        )

    # Compute objective
    objective_value = sum(d[i]^2 + d[i] for i in chosen)

    # Store objective
    push!(objectives, objective_value)

    # =======================================================
    #  ESTIMATE PROBABILISTIC FEASIBILITY (Monte Carlo)
    # -------------------------------------------------------
    # Check how often:
    #    sum(W[i,t] * x[i]) >= percent_demand * D[t]
    # holds for *random draws* of W and D.
    # =======================================================

    feasible_count = 0

    for trial in 1:N_MC
        
        # Sample demand & traffic from normal distributions
        D_sample = D_mean_robust .+ D_std_robust .* randn(length(D_mean_robust))
        W_sample = W_mean_robust .+ W_std_robust .* randn(size(W_mean_robust))

        # Check feasibility for all days
        is_feasible = true
        for t in 1:length(D_mean_robust)
            energy = sum(W_sample[i,t] * watts_converted for i in chosen)
            if energy < percent_demand * D_sample[t]
                is_feasible = false
                break
            end
        end

        feasible_count += is_feasible ? 1 : 0
    end

    prob = feasible_count / N_MC
    push!(prob_feasible, prob)
end

println("\nDone!\n")

In [ ]:
println(objectives)
println(prob_feasible)

In [ ]:
using Plots
using Statistics

# Range of k to test
k_values = 0:0.2:2.4

# Storage vectors
objectives_global = Float64[]
prob_feasible_global = Float64[]

# Monte Carlo samples for feasibility estimation
N_MC = 2000

println("Running global robust optimization sweep...\n")

for k in k_values
    println("Testing global robust k = ", k)

    # === RUN GLOBAL ROBUST MODEL ===
    num_locs, chosen, total_dist, model =
        optimize_installations_dist_robust_global(
            D_mean_robust, D_std_robust,
            W_mean_robust, W_std_robust,
            d,
            k,
            1000
        )

    # === OBJECTIVE VALUE ===
    obj_val = sum(d[i]^2 + d[i] for i in chosen)
    push!(objectives_global, obj_val)

    # === PROBABILISTIC FEASIBILITY ESTIMATION ===
    feas_count = 0

    for trial in 1:N_MC
        # Sample demand
        D_sample = D_mean_robust .+ D_std_robust .* randn(length(D_mean_robust))

        # Sample traffic
        W_sample = W_mean_robust .+ W_std_robust .* randn(size(W_mean_robust))

        # Check feasibility for all days
        feasible = true
        for t in 1:length(D_mean_robust)
            energy = sum(W_sample[i,t] * watts_converted for i in chosen)
            if energy < percent_demand * D_sample[t]
                feasible = false
                break
            end
        end
        feas_count += feasible ? 1 : 0
    end

    push!(prob_feasible_global, feas_count / N_MC)
end

println("\nGlobal robust sweep finished.\n")

In [ ]:
#nominal model
objectives_nominal = Float64[]
prob_feasible_nominal = Float64[]

# Solve nominal model ONCE
num_nom, chosen_nom, dist_nom, model_nom =
    optimize_installations_dist(D_mean_robust, W_mean_robust, d, 1000)

# Calculate objective value
obj_nominal = sum(d[i]^2 + d[i] for i in chosen_nom)

# Repeat the same objective for each k (horizontal line)
objectives_nominal = fill(obj_nominal, length(k_values))

# --- Monte Carlo Feasibility ---
N_MC = 2000
feas_nom = 0

for trial in 1:N_MC
    D_sample = D_mean_robust .+ D_std_robust .* randn(length(D_mean_robust))
    W_sample = W_mean_robust .+ W_std_robust .* randn(size(W_mean_robust))

    feasible = true
    for t in 1:length(D_sample)
        energy = sum(W_sample[i,t] * watts_converted for i in chosen_nom)
        if energy < percent_demand * D_sample[t]
            feasible = false
            break
        end
    end

    feas_nom += feasible ? 1 : 0
end

prob_feasible_nominal = fill(feas_nom / N_MC, length(k_values))


In [ ]:
p_obj_all = plot(
    k_values, objectives_nominal,
    label="Nominal", linewidth=2
)

plot!(p_obj_all,
    k_values, objectives,
    label="Robust 1 (Element)", markershape=:circle
)

plot!(p_obj_all,
    k_values, objectives_global,
    label="Robust 2 (Global)", markershape=:diamond
)

xlabel!("Γ (k)")
ylabel!("Objective")
title!("Objective vs Robustness Parameter")
display(p_obj_all)


In [ ]:
p_feas_all = plot(
    k_values, prob_feasible_nominal,
    label="Nominal", linewidth=2, ylim=(0,1)
)

plot!(p_feas_all,
    k_values, prob_feasible,
    label="Robust 1 (Element)", markershape=:circle
)

plot!(p_feas_all,
    k_values, prob_feasible_global,
    label="Robust 2 (Global)", markershape=:diamond
)

xlabel!("Γ (k)")
ylabel!("Probabilistic Feasibility")
title!("Probabilistic Feasibility vs Robustness Parameter")
display(p_feas_all)
